# Spark Operations using Spark DataFrames and Spark SQL

### In this activity we will understand
-  What are DataFrames in Spark ?
-  Different ways to create a DataFrames
-  What are Spark Transformations & Actions
-  Verify Summary Statistics
-  Spark SQL
-  Performance Comparison of Spark DataFrame and Spark SQL
-  Column References
-  Converting to Spark Types - Literals
-  Add/Rename/Remove Columns
-  TypeCasting
-  Column differences
-  Pair-wise frequencies
-  Remove duplicates
-  Working with Nulls
-  Filtering the rows
-  Aggregations
-  Joins
-  Random Samples
-  Random Splits
-  Map Transformations
-  Sorting
-  Union
-  String Manipulations
-  Regular Expressions
-  Working with Dates and Time Stamp
-  User Defined Functions 
-  Broadcase variables and Accumulators
-  Handling Different Data Sources
-  New features available in Spark 2.3 
    -  PySpark Usage for Pandas with Apache Arrow
        -  Enabling for Conversion to/from Pandas
        -  Pandas UDFs (a.k.a. Vectorized UDFs)
            - Scalar
            - Grouped Map

In [1]:
## Set Python - Spark environment.
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.6-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [2]:
## Create SparkContext, SparkSession
from os.path import expanduser, join, abspath

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs:///apps/hive/warehouse/'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext    

In [3]:
spark

In [4]:
sc

<SparkContext master=local[*] appName=Python Spark SQL Hive integration example>

## Data Representation
- **Pandas** - DataFrames represented on a single machine as Python data structures
- **RDDs** - Spark’s foundational structure Resilient Distributed Dataset is represented as a reference to partitioned data without types
- **DataFrames** - Spark’s strongly typed optimized distributed collection of rows

## ** Spark DataFrame **

#### A DataFrame is the most common Structured API and simply represents a table of data with rows and columns. 
<br> The list that defines the columns and the types within those columns is called the schema. 
<br> One can think of a DataFrame as a spreadsheet with named columns.
<br> A spreadsheet sits on one computer in one specific location, whereas a Spark DataFrame can span thousands of computers.
<br> The reason for putting the data on more than one computer should be intuitive: 
<br>     either the data is too large to fit on one machine or 
<br>     it would simply take too long to perform that computation on one machine.

#### NOTE
Spark has several core abstractions: Datasets, DataFrames, SQL Tables, and Resilient Distributed Datasets (RDDs). 
<br> These different abstractions all represent distributed collections of data. 
<br> The easiest and most efficient are DataFrames, which are available in all languages.

![Spark DataFrame](./Images/SparkDataFrame.png)

#### Create a dataframe with one column containing 100 rows with values from 0 to 99.
This range of numbers represents a distributed collection. 
<br> When run on a cluster, each part of this range of numbers exists on a different executor. 
<br> This is a Spark DataFrame.

In [5]:
myRange = spark.range(100).toDF('number')

In [6]:
myRange

DataFrame[number: bigint]

In [7]:
myRange.show(4)

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
+------+
only showing top 4 rows



In [8]:
myDF = spark.createDataFrame([[1, 'Alice', 30],
                              [2, 'Bob', 28],
                              [3, 'Cathy', 31], 
                              [4, 'Dave', 56]], ['Id', 'Name', 'Age'])

myDF.show()

+---+-----+---+
| Id| Name|Age|
+---+-----+---+
|  1|Alice| 30|
|  2|  Bob| 28|
|  3|Cathy| 31|
|  4| Dave| 56|
+---+-----+---+



## DataFrame Transformations & Actions

### Transformations
In Spark, the core data structures are immutable, meaning they cannot be changed after they’re created.
<br> To “change” a DataFrame, you need to instruct Spark how you would like to modify it to do what you want.
<br> These instructions are called transformations.
<br> Transformations are the core of how you express your business logic using Spark.
<br> Transformations are simply ways of specifying different series of data manipulation.

![Spark Transformations](./Images/Spark_Transformations.png)

In [9]:
divisBy2 = myRange.where("number % 2 = 0")
divisBy2

DataFrame[number: bigint]

Notice that these return no output. <br>This is because we specified only an abstract transformation, and Spark will not act on transformations until we call an action.

### Actions
Transformations allow us to build up our logical transformation plan. 
<br> To trigger the computation, we run an action.
<br> An action instructs Spark to compute a result from a series of transformations. 
<br> The simplest action is count, which gives us the total number of records in the DataFrame:

#### There are 3 types of actions
Actions to view data in the console
<br>Actions to collect data to native objects in the respective language
<br>Actions to write to output data sources

In [10]:
divisBy2.count()

50

### Interoperating with RDDs

<br> Spark SQL supports two different methods for converting existing RDDs into DataFrames. 
<br> The first method uses reflection to infer the schema of an RDD that contains specific types of objects. 
<br> This reflection based approach leads to more concise code and works well when you already know the schema while writing your Spark application.

<br> The second method for creating DataFrames is through a programmatic interface that allows you to construct a schema and then apply it to an existing RDD. 
<br> While this method is more verbose, it allows you to construct Datasets when the columns and their types are not known until runtime.

In [11]:
sc = spark.sparkContext

#### Inferring the Schema Using Reflection

In [12]:
!pwd

/home/insofe/IBM/Day03


In [13]:
# Create an RDD from a source
tempRDD = sc.textFile("file:///home/insofe/IBM/Day03/temp_data.txt") 

In [14]:
tempRDD.take(3)

[u'1901\t-78\t1', u'1901\t-72\t1', u'1901\t-94\t1']

In [15]:
splitRDD = tempRDD.map(lambda line: line.split("\t"))
splitRDD.take(3)

[[u'1901', u'-78', u'1'], [u'1901', u'-72', u'1'], [u'1901', u'-94', u'1']]

In [16]:
schemafiedRDD = splitRDD.map(lambda line: Row(year=line[0], temp=int(line[1]), status=int(line[2])))
schemafiedRDD.take(3)

[Row(status=1, temp=-78, year=u'1901'),
 Row(status=1, temp=-72, year=u'1901'),
 Row(status=1, temp=-94, year=u'1901')]

In [17]:
# Infer the schema, and register the DataFrame as a table.
tempDF = spark.createDataFrame(schemafiedRDD)
tempDF.show(3)

+------+----+----+
|status|temp|year|
+------+----+----+
|     1| -78|1901|
|     1| -72|1901|
|     1| -94|1901|
+------+----+----+
only showing top 3 rows



#### Programmatically Specifying the Schema
- Create an RDD of tuples or lists from the original RDD;
- Create the schema represented by a StructType matching the structure of tuples or lists in the RDD created in the step 1.
- Apply the schema to the RDD via createDataFrame method provided by SparkSession.

In [18]:
testRDD = sc.textFile("file:///home/insofe/IBM/Day03/test_sample10.csv")
print("Total Records with header: ", testRDD.count())
print("\nFirst Two Records Before Removing Header\n")
print(testRDD.take(2))

('Total Records with header: ', 23380)

First Two Records Before Removing Header

[u'User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3', u'1000029,P00111542,M,36-45,7,C,1,0,2,17,']


In [19]:
header = testRDD.first()
testRDD = testRDD.filter(lambda line: line != header)
print("Total Records without header: ", testRDD.count())
print("\nFirst Two Records After Removing Header\n")
print(testRDD.take(2))

('Total Records without header: ', 23379)

First Two Records After Removing Header

[u'1000029,P00111542,M,36-45,7,C,1,0,2,17,', u'1000034,P00265242,F,18-25,0,A,0,0,5,8,']


In [20]:
# Split the data into individual columns
splitRDD = testRDD.map(lambda line: line.split(","))
print("\nFirst Two Records After Split/Parsing\n")
print(splitRDD.take(2))


First Two Records After Split/Parsing

[[u'1000029', u'P00111542', u'M', u'36-45', u'7', u'C', u'1', u'0', u'2', u'17', u''], [u'1000034', u'P00265242', u'F', u'18-25', u'0', u'A', u'0', u'0', u'5', u'8', u'']]


#### Create a dataframe for the above Data
1. Define Schema
2. Create dataframe using the above schema

#### Create Schema

In [21]:
from pyspark.sql.types import *

testSchema = StructType([
    StructField("User_ID", StringType(), True),
    StructField("Product_ID", StringType(), True),
    StructField("Gender", StringType(), True),
    StructField("Age", StringType(), True),
    StructField("Occupation", StringType(), True),
    StructField("City_Category", StringType(), True),
    StructField("Stay_In_Current_City_Years", StringType(), True),
    StructField("Marital_Status", StringType(), True),
    StructField("Product_Category_1", StringType(), True),
    StructField("Product_Category_2", StringType(), True),
    StructField("Product_Category_3", StringType(), True)
])

#### Create DataFrame using the above schema

In [22]:
testDF = spark.createDataFrame(data = splitRDD, schema=testSchema)

In [23]:
testDF.take(2)

[Row(User_ID=u'1000029', Product_ID=u'P00111542', Gender=u'M', Age=u'36-45', Occupation=u'7', City_Category=u'C', Stay_In_Current_City_Years=u'1', Marital_Status=u'0', Product_Category_1=u'2', Product_Category_2=u'17', Product_Category_3=u''),
 Row(User_ID=u'1000034', Product_ID=u'P00265242', Gender=u'F', Age=u'18-25', Occupation=u'0', City_Category=u'A', Stay_In_Current_City_Years=u'0', Marital_Status=u'0', Product_Category_1=u'5', Product_Category_2=u'8', Product_Category_3=u'')]

In [24]:
testDF.show(4)

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|1000029| P00111542|     M|36-45|         7|            C|                         1|             0|                 2|                17|                  |
|1000034| P00265242|     F|18-25|         0|            A|                         0|             0|                 5|                 8|                  |
|1000053|  P0097342|     M|26-35|         0|            B|                         1|             0|                 1|                15|                16|
|1000080| P00112142|     M|  55+|         1|        

In [25]:
testRDD.count()

23379

In [26]:
testRDD.take(2)

[u'1000029,P00111542,M,36-45,7,C,1,0,2,17,',
 u'1000034,P00265242,F,18-25,0,A,0,0,5,8,']

Reading a CSV file into a DataFrame and converting it to a local array or list of rows.


![Reading CSV](./Images/csvDataFrame.png)

In [27]:
## Read data and create the dataframes
## trainDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("file:///home/rameshm/Datasets/SalesData/train.csv")
## testDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("file:///home/rameshm/Datasets/SalesData/test.csv")
trainDF = spark.read.format("csv")\
        .option("header", "true")\
        .option("inferSchema", "true")\
        .load("file:///home/insofe/IBM/Day03/train_sample10.csv")
        
# testDF = spark.read.format("csv")\
#         .option("header", "true")\
#         .option("inferSchema", "true")\
#         .load("file:///home/rameshm/Datasets/SalesData/test.csv")

In [28]:
trainDF.take(5)

[Row(User_ID=1000001, Product_ID=u'P00069042', Gender=u'F', Age=u'0-17', Occupation=10, City_Category=u'A', Stay_In_Current_City_Years=u'2', Marital_Status=0, Product_Category_1=3, Product_Category_2=None, Product_Category_3=None, Purchase=8370),
 Row(User_ID=1000001, Product_ID=u'P00248942', Gender=u'F', Age=u'0-17', Occupation=10, City_Category=u'A', Stay_In_Current_City_Years=u'2', Marital_Status=0, Product_Category_1=1, Product_Category_2=6.0, Product_Category_3=14.0, Purchase=15200),
 Row(User_ID=1000001, Product_ID=u'P00087842', Gender=u'F', Age=u'0-17', Occupation=10, City_Category=u'A', Stay_In_Current_City_Years=u'2', Marital_Status=0, Product_Category_1=12, Product_Category_2=None, Product_Category_3=None, Purchase=1422),
 Row(User_ID=1000001, Product_ID=u'P00085442', Gender=u'F', Age=u'0-17', Occupation=10, City_Category=u'A', Stay_In_Current_City_Years=u'2', Marital_Status=0, Product_Category_1=12, Product_Category_2=14.0, Product_Category_3=None, Purchase=1057),
 Row(User_

#### Verify Schema

In [29]:
## Print Schema
trainDF.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: double (nullable = true)
 |-- Product_Category_3: double (nullable = true)
 |-- Purchase: integer (nullable = true)



In [30]:
## To Show first n observations
## Use head operation to see first n observations (say, 2 observations). 
## Head operation in PySpark is similar to head operation in Pandas.
trainDF.head(2)

[Row(User_ID=1000001, Product_ID=u'P00069042', Gender=u'F', Age=u'0-17', Occupation=10, City_Category=u'A', Stay_In_Current_City_Years=u'2', Marital_Status=0, Product_Category_1=3, Product_Category_2=None, Product_Category_3=None, Purchase=8370),
 Row(User_ID=1000001, Product_ID=u'P00248942', Gender=u'F', Age=u'0-17', Occupation=10, City_Category=u'A', Stay_In_Current_City_Years=u'2', Marital_Status=0, Product_Category_1=1, Product_Category_2=6.0, Product_Category_3=14.0, Purchase=15200)]

In [31]:
## Above results are comprised of row like format. 
## To see the result in more interactive manner (rows under the columns), Use the show operation. 
## Show operation on train and take first 5 rows of it. 
trainDF.show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
only

In [32]:
## To Count the number of rows in DataFrame
print('Total records count in train dataset is {}'.format(trainDF.count()))
print('Total records count in test dataset is {}'.format(testDF.count()))

Total records count in train dataset is 550068
Total records count in test dataset is 23379


In [33]:
## Columns count and column names
print("Total Columns count in train dataset is {}".format(len(trainDF.columns)))
print("\n\nColumns in train dataset are: {} \n".format(trainDF.columns))

print("Total Columns count in test dataset is {}".format(len(testDF.columns)))
print("\n\nColumns in test dataset are: {} \n".format(testDF.columns))

Total Columns count in train dataset is 12


Columns in train dataset are: ['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Purchase'] 

Total Columns count in test dataset is 11


Columns in test dataset are: ['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3'] 



#### Summary statistics

In [34]:
## To get the summary statistics (mean, standard deviance, min ,max , count) of numerical columns in a DataFrame
trainDF.describe().show()

+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|       Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|            550068|    550068|550068|550068|           550068|       550068|                    550068|             550068|            550068|            376430|            166821|           550068|
|   mean|1003028.8424013031|      null|  null|  null|8.076706879876669|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 9.84232925112238

In [35]:
## Check what happens when we specify the name of a categorical / String columns in describe operation.
## describe operation is working for String type column but the output for mean, stddev are null and 
## min & max values are calculated based on ASCII value of categories.
trainDF.describe('Purchase').show()

+-------+-----------------+
|summary|         Purchase|
+-------+-----------------+
|  count|           550068|
|   mean|9263.968712959126|
| stddev|5023.065393820567|
|    min|               12|
|    max|            23961|
+-------+-----------------+



### Spark SQL
With Spark SQL, you can register any DataFrame as a table or view (a temporary table) and query it using pure SQL. 
<br>There is no performance difference between writing SQL queries or writing DataFrame code, <br>they both “compile” to the same underlying plan that we specify in DataFrame code.

In [36]:
## Create view/table
trainDF.createOrReplaceTempView("trainDFTable")

In [37]:
## Verify Dataframe
trainDF.show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
only

In [38]:
## Verify Dataframe
trainDF.take(2)

[Row(User_ID=1000001, Product_ID=u'P00069042', Gender=u'F', Age=u'0-17', Occupation=10, City_Category=u'A', Stay_In_Current_City_Years=u'2', Marital_Status=0, Product_Category_1=3, Product_Category_2=None, Product_Category_3=None, Purchase=8370),
 Row(User_ID=1000001, Product_ID=u'P00248942', Gender=u'F', Age=u'0-17', Occupation=10, City_Category=u'A', Stay_In_Current_City_Years=u'2', Marital_Status=0, Product_Category_1=1, Product_Category_2=6.0, Product_Category_3=14.0, Purchase=15200)]

In [39]:
## Verify Table
spark.sql("SELECT * FROM trainDFTable LIMIT 2").show()

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



#### Performance Comparison Spark DataFrame vs Spark SQL

In [40]:
#dataframeWay = trainDF.where(trainDF.Purchase>15000).count()
dataframeWay = trainDF.groupBy('Age').count()
dataframeWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[Age#121], functions=[count(1)])
+- Exchange hashpartitioning(Age#121, 200)
   +- *(1) HashAggregate(keys=[Age#121], functions=[partial_count(1)])
      +- *(1) FileScan csv [Age#121] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/insofe/IBM/Day03/train_sample10.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Age:string>


In [41]:
sqlWay = spark.sql("SELECT Age, count(1) FROM trainDFTable GROUP BY Age")
sqlWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[Age#121], functions=[count(1)])
+- Exchange hashpartitioning(Age#121, 200)
   +- *(1) HashAggregate(keys=[Age#121], functions=[partial_count(1)])
      +- *(1) FileScan csv [Age#121] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/insofe/IBM/Day03/train_sample10.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Age:string>


#### Column References

#### Select & SelectExpr

In [42]:
## Multiple ways of referring a column in a dataframe
from pyspark.sql.functions import expr, col, column

trainDF.select(expr("User_ID AS userID") , 
               col("User_ID"), 
               column("User_ID"), "User_ID").show(2)

+-------+-------+-------+-------+
| userID|User_ID|User_ID|User_ID|
+-------+-------+-------+-------+
|1000001|1000001|1000001|1000001|
|1000001|1000001|1000001|1000001|
+-------+-------+-------+-------+
only showing top 2 rows



In [43]:
trainDF.select(col("User_ID"), "User_ID")

DataFrame[User_ID: int, User_ID: int]

#### Pandas dot notation doesn't work here 

In [44]:
result = trainDF.User_ID

This will save/assign a column name to the newly created variable

In [45]:
# select content from the above column
trainDF.select(result).show(2)

+-------+
|User_ID|
+-------+
|1000001|
|1000001|
+-------+
only showing top 2 rows



In [46]:
trainDF.select(expr("User_ID AS userID")).show(2)

+-------+
| userID|
+-------+
|1000001|
|1000001|
+-------+
only showing top 2 rows



In [47]:
spark.sql("SELECT User_ID AS userID FROM trainDFTable").show(2)

+-------+
| userID|
+-------+
|1000001|
|1000001|
+-------+
only showing top 2 rows



In [48]:
trainDF.selectExpr("User_ID AS userID", "Product_ID AS productID").show(2)

+-------+---------+
| userID|productID|
+-------+---------+
|1000001|P00069042|
|1000001|P00248942|
+-------+---------+
only showing top 2 rows



In [49]:
trainDF.select("User_ID", "Product_ID", "Age").show(2)

+-------+----------+----+
|User_ID|Product_ID| Age|
+-------+----------+----+
|1000001| P00069042|0-17|
|1000001| P00248942|0-17|
+-------+----------+----+
only showing top 2 rows



#### Converting to Spark Types (Literals)
Sometimes we need to pass explicit values into Spark that aren’t a new column but are just a value in all the rows. This might be a constant value or something we’ll need to compare to later on. The way we do this is through literals. 
This is basically a translation from a given programming language’s literal value to one that Spark understands. 
Literals are expressions and can be used in the same way.

In [50]:
from pyspark.sql.functions import lit
trainDF.select("*", lit(1).alias('One')).show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|One|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|  1|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|  1|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+-------------

In [51]:
## In SQL, literals are just the specific value.
spark.sql("SELECT *, 1 as One FROM trainDFTable LIMIT 2").show()

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|One|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|  1|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|  1|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+-------------

#### Adding Columns

In [52]:
## More Formal way
trainDF.withColumn("One", lit(1)).show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|One|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|  1|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|  1|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+-------------

In [53]:
spark.sql("SELECT *, 1 AS One FROM trainDFTable LIMIT 2").show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|One|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|  1|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|  1|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+-------------

In [54]:
tempDF = trainDF.withColumn("SameCategoryCode", 
trainDF["Product_Category_1"] == trainDF["Product_Category_2"])
tempDF.show(4)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|SameCategoryCode|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|            null|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|           false|
|1000001| P00087842|     F|0-17|        10|            A|                         2| 

#### Renaming Columns

In [55]:
tempDF.withColumnRenamed("SameCategoryCode", "SimilarCategory").show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---------------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|SimilarCategory|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+---------------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|           null|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|          false|
+-------+----------+------+----+----------+-------------+--------------------------+------

#### Removing Columns

In [56]:
tempDF.drop("SameCategoryCode").show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
only

#### Changing a Column’s Type (cast)

In [57]:
tempDF.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: double (nullable = true)
 |-- Product_Category_3: double (nullable = true)
 |-- Purchase: integer (nullable = true)
 |-- SameCategoryCode: boolean (nullable = true)



In [58]:
tempDF.withColumn("Purchase", col("Purchase").cast("string")).printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: double (nullable = true)
 |-- Product_Category_3: double (nullable = true)
 |-- Purchase: string (nullable = true)
 |-- SameCategoryCode: boolean (nullable = true)



#### Distinct Values

In [59]:
## To find the number of distinct product in train and test datasets
## To calculate the number of distinct products in train and test datasets apply distinct operation.
print("Distinct values in Product_ID's in train dataset are {}".format(trainDF.select('Product_ID').distinct().count()))
print("Distinct values in Product_ID's in test dataset are {}".format(testDF.select('Product_ID').distinct().count()))

Distinct values in Product_ID's in train dataset are 3631
Distinct values in Product_ID's in test dataset are 2762


#### Differences in two columns

In [60]:
## From the above we can see the train file has more categories than test file. 
## Let us check what are the categories for Product_ID, which are in test file but not in train file by 
## applying subtract operation.
## We can do the same for all categorical features.
diff_cat_in_test_train=testDF.select('Product_ID').subtract(trainDF.select('Product_ID'))
print("Count of Product_ID's there in test dataset but not train dataset are {}".format(diff_cat_in_test_train.count()))

diff_cat_in_train_test=trainDF.select('Product_ID').subtract(testDF.select('Product_ID'))
print("Count of Product_ID's there in train dataset but not test dataset are {}".format(diff_cat_in_train_test.count()))

Count of Product_ID's there in test dataset but not train dataset are 5
Count of Product_ID's there in train dataset but not test dataset are 874


#### Pair wise Frequencies - Crosstab

In [61]:
## To calculate pair wise frequency of categorical columns
## Use crosstab operation on DataFrame to calculate the pair wise frequency of columns. 
## Apply crosstab operation on ‘Age’ and ‘Gender’ columns of train DataFrame.
trainDF.crosstab('Age', 'Gender').show()

+----------+-----+------+
|Age_Gender|    F|     M|
+----------+-----+------+
|      0-17| 5083| 10019|
|     46-50|13199| 32502|
|     18-25|24628| 75032|
|     36-45|27170| 82843|
|       55+| 5083| 16421|
|     51-55| 9894| 28607|
|     26-35|50752|168835|
+----------+-----+------+



In [62]:
trainDF.groupBy('Age', 'Gender').count().show()

+-----+------+------+
|  Age|Gender| count|
+-----+------+------+
|51-55|     F|  9894|
|18-25|     M| 75032|
| 0-17|     F|  5083|
|46-50|     M| 32502|
|18-25|     F| 24628|
|  55+|     M| 16421|
|  55+|     F|  5083|
|36-45|     M| 82843|
|26-35|     F| 50752|
| 0-17|     M| 10019|
|36-45|     F| 27170|
|51-55|     M| 28607|
|26-35|     M|168835|
|46-50|     F| 13199|
+-----+------+------+



In [63]:
spark.sql("""select Age,
    sum(case when Gender = 'F' then 1 else 0 end) F,
    sum(case when Gender = 'M' then 1 else 0 end) M
from trainDFTable
group by Age""").show()

# spark.sql("""select Age,
#     count(*) total,
#     sum(case when Gender = 'F' then 1 else 0 end) F,
#     sum(case when Gender = 'M' then 1 else 0 end) M
# from trainDFTable
# group by Age""").show()

+-----+-----+------+
|  Age|    F|     M|
+-----+-----+------+
|18-25|24628| 75032|
|26-35|50752|168835|
| 0-17| 5083| 10019|
|46-50|13199| 32502|
|51-55| 9894| 28607|
|36-45|27170| 82843|
|  55+| 5083| 16421|
+-----+-----+------+



#### Removing Duplicates

In [64]:
##To get the DataFrame without any duplicate rows of given a DataFrame
##Use dropDuplicates operation to drop the duplicate rows of a DataFrame. 
## In this command, performing this on two columns Age and Gender of train dataset and 
## Get the all unique rows for these two columns.
trainDF.select('Age','Gender').dropDuplicates().show()

+-----+------+
|  Age|Gender|
+-----+------+
|51-55|     F|
|18-25|     M|
| 0-17|     F|
|46-50|     M|
|18-25|     F|
|  55+|     M|
|  55+|     F|
|36-45|     M|
|26-35|     F|
| 0-17|     M|
|36-45|     F|
|51-55|     M|
|26-35|     M|
|46-50|     F|
+-----+------+



#### Working with Nulls in Data

In [65]:
## To drop the all rows with null value?
## Use dropna operation. 
## To drop row from the DataFrame it consider three options.
## how – ‘any’ or ‘all’. If ‘any’, drop a row if it contains any nulls. If ‘all’, drop a row only if all its values are null.

## thresh – int, default None If specified, drop rows that have less than thresh non-null values. 
## This overwrites the how parameter.

## subset – optional list of column names to consider.

##Drop null rows in train with default parameters and count the rows in output DataFrame. 
## Default options are any, None, None for how, thresh, subset respectively.
print(trainDF.dropna().count())
print(trainDF.na.drop().count())
print(trainDF.na.drop("any").count())

166821
166821
166821


In [66]:
## To replace the null values in DataFrame with constant number
## Use fillna operation. 

##The fillna will take two parameters to fill the null values.
## value:
##     It will take a dictionary to specify which column will replace with which value.
##     A value (int , float, string) for all columns.
##subset: Specify some selected columns.

##Fill ‘-1’ inplace of null values in train DataFrame.
trainDF.fillna(-1).show(5)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              -1.0|              -1.0|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|               6.0|              14.0|   15200|
|1000001| P00087842|     F|0-17|        10|            A|                         2|             0|                12|              -1.0|              -1.0|    1422|
|100

In [67]:
## Filling with different values for different columns
fill_cols_vals = {
"Gender": 'M',
"Purchase" : 999999
}
trainDF.na.fill(fill_cols_vals).count()

550068

In [68]:
trainDF.na.replace([""], ["UNKNOWN"], "Gender").count()

550068

#### Filtering the rows

In [69]:
## To filter the rows in train dataset which has Purchases more than 15000
## apply the filter operation on Purchase column in train DataFrame 
## to filter out the rows with values more than 15000. 
print("Count of rows where Purchase Amount more than 15000 are {}".format(trainDF.filter(trainDF.Purchase > 15000).count()))
print("Count of rows where Purchase Amount more than 15000 are {}".format(trainDF.filter(col("Purchase") > 15000).count()))
print("Count of rows where Purchase Amount more than 15000 are {}".format(trainDF.filter(column("Purchase") > 15000).count()))
print("Count of rows where Purchase Amount more than 15000 are {}".format(trainDF.filter(expr("Purchase") > 15000).count()))
print("Count of rows where Purchase Amount more than 15000 are {}".format(trainDF.filter(trainDF["Purchase"] > 15000).count()))

Count of rows where Purchase Amount more than 15000 are 110523
Count of rows where Purchase Amount more than 15000 are 110523
Count of rows where Purchase Amount more than 15000 are 110523
Count of rows where Purchase Amount more than 15000 are 110523
Count of rows where Purchase Amount more than 15000 are 110523


In [70]:
spark.sql("""
SELECT 
COUNT(*) AS Count
FROM trainDFTable
WHERE Purchase > 15000""").show()

+------+
| Count|
+------+
|110523|
+------+



In [71]:
trainDF.where("Purchase > 15000").where("Gender = 'F'").count()

21429

In [72]:
trainDF.filter("Purchase > 15000").where("Gender = 'F'").count()

21429

In [73]:
trainDF.where((col("Purchase") > 15000) & (col("Gender") == 'F')).count()

21429

In [74]:
trainDF.filter((col("Purchase") > 15000) & (col("Gender") == 'F')).count()

21429

In [75]:
spark.sql("SELECT * FROM trainDFTable WHERE Purchase > 15000 AND Gender = 'F'").count()

21429

## Aggregations

#### Count Distinct

In [76]:
from pyspark.sql.functions import countDistinct
trainDF.select(countDistinct("Age")).show()

+-------------------+
|count(DISTINCT Age)|
+-------------------+
|                  7|
+-------------------+



#### Approximate Count Distinct

In [77]:
from pyspark.sql.functions import approx_count_distinct
trainDF.select(approx_count_distinct("Age", 0.1)).show()

+--------------------------+
|approx_count_distinct(Age)|
+--------------------------+
|                         7|
+--------------------------+



#### First and Last

In [78]:
from pyspark.sql.functions import first, last
trainDF.select(first("Product_ID"), last("Product_ID")).show()

+------------------------+-----------------------+
|first(Product_ID, false)|last(Product_ID, false)|
+------------------------+-----------------------+
|               P00069042|              P00371644|
+------------------------+-----------------------+



#### Min and Max

In [79]:
from pyspark.sql.functions import min, max
trainDF.select(min("Purchase"), max("Purchase")).show()

+-------------+-------------+
|min(Purchase)|max(Purchase)|
+-------------+-------------+
|           12|        23961|
+-------------+-------------+



#### Sum

In [80]:
from pyspark.sql.functions import sum
trainDF.select(sum("Purchase")).show()

+-------------+
|sum(Purchase)|
+-------------+
|   5095812742|
+-------------+



#### sumDistinct

In [81]:
from pyspark.sql.functions import sumDistinct
trainDF.select(sumDistinct("Purchase")).show()

+----------------------+
|sum(DISTINCT Purchase)|
+----------------------+
|             208520914|
+----------------------+



#### Avg

In [82]:
from pyspark.sql.functions import sum, count, avg, expr

trainDF.select(
    count("Purchase").alias("total_transactions"),
    sum("Purchase").alias("total_purchases"),
    avg("Purchase").alias("avg_purchases"),
    expr("mean(Purchase)").alias("mean_purchases"))\
  .selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases").show()

+--------------------------------------+-----------------+-----------------+
|(total_purchases / total_transactions)|    avg_purchases|   mean_purchases|
+--------------------------------------+-----------------+-----------------+
|                     9263.968712959126|9263.968712959126|9263.968712959126|
+--------------------------------------+-----------------+-----------------+



#### Variance and Standard Deviation

In [83]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp

trainDF.select(var_pop("Purchase"), var_samp("Purchase"),
  stddev_pop("Purchase"), stddev_samp("Purchase")).show()

+-------------------+-------------------+--------------------+---------------------+
|  var_pop(Purchase)| var_samp(Purchase)|stddev_pop(Purchase)|stddev_samp(Purchase)|
+-------------------+-------------------+--------------------+---------------------+
|2.523114008138533E7|2.523118595059777E7|   5023.060827959914|    5023.065393820567|
+-------------------+-------------------+--------------------+---------------------+



In [84]:
spark.sql("""SELECT var_pop(Purchase), var_samp(Purchase),
             stddev_pop(Purchase), stddev_samp(Purchase)
             FROM trainDFTable""").show()

+---------------------------------+----------------------------------+------------------------------------+-------------------------------------+
|var_pop(CAST(Purchase AS DOUBLE))|var_samp(CAST(Purchase AS DOUBLE))|stddev_pop(CAST(Purchase AS DOUBLE))|stddev_samp(CAST(Purchase AS DOUBLE))|
+---------------------------------+----------------------------------+------------------------------------+-------------------------------------+
|              2.523114008138533E7|               2.523118595059777E7|                   5023.060827959914|                    5023.065393820567|
+---------------------------------+----------------------------------+------------------------------------+-------------------------------------+



#### skewness and kurtosis

In [85]:
from pyspark.sql.functions import skewness, kurtosis
trainDF.select(skewness("Purchase"), kurtosis("Purchase")).show()

+------------------+--------------------+
|skewness(Purchase)|  kurtosis(Purchase)|
+------------------+--------------------+
|0.6001383671643401|-0.33838539753600205|
+------------------+--------------------+



In [86]:
spark.sql("""SELECT skewness(Purchase), kurtosis(Purchase)
             FROM trainDFTable""").show()

+----------------------------------+----------------------------------+
|skewness(CAST(Purchase AS DOUBLE))|kurtosis(CAST(Purchase AS DOUBLE))|
+----------------------------------+----------------------------------+
|                0.6001383671643401|              -0.33838539753600205|
+----------------------------------+----------------------------------+



#### Covariance and Correlation

In [87]:
from pyspark.sql.functions import corr, covar_pop, covar_samp
trainDF.select(corr("Product_Category_1", "Purchase"), covar_samp("Product_Category_1", "Purchase"),
    covar_pop("Product_Category_1", "Purchase")).show()

+----------------------------------+----------------------------------------+---------------------------------------+
|corr(Product_Category_1, Purchase)|covar_samp(Product_Category_1, Purchase)|covar_pop(Product_Category_1, Purchase)|
+----------------------------------+----------------------------------------+---------------------------------------+
|              -0.34370334591990787|                     -6795.6500072045255|                     -6795.637653004668|
+----------------------------------+----------------------------------------+---------------------------------------+



In [88]:
spark.sql("""SELECT corr(Product_Category_1, Purchase), covar_samp(Product_Category_1, Purchase),
             covar_pop(Product_Category_1, Purchase)
             FROM trainDFTable""").show()

+------------------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------------------------------------------+
|corr(CAST(Product_Category_1 AS DOUBLE), CAST(Purchase AS DOUBLE))|covar_samp(CAST(Product_Category_1 AS DOUBLE), CAST(Purchase AS DOUBLE))|covar_pop(CAST(Product_Category_1 AS DOUBLE), CAST(Purchase AS DOUBLE))|
+------------------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------------------------------------------+
|                                              -0.34370334591990787|                                                     -6795.6500072045255|                                                     -6795.637653004668|
+------------------------------------------------------------------+------------------------------------------------------------------------+---

#### Complex Aggregations

In [89]:
from pyspark.sql.functions import collect_set, collect_list
trainDF.agg(collect_set("Age"), collect_list("Age")).show()

+--------------------+--------------------+
|    collect_set(Age)|   collect_list(Age)|
+--------------------+--------------------+
|[55+, 51-55, 0-17...|[0-17, 0-17, 0-17...|
+--------------------+--------------------+



In [90]:
spark.sql("""SELECT collect_set(Age), collect_list(Age) FROM trainDFTable""").show()

+--------------------+--------------------+
|    collect_set(Age)|   collect_list(Age)|
+--------------------+--------------------+
|[55+, 51-55, 0-17...|[0-17, 0-17, 0-17...|
+--------------------+--------------------+



#### Grouping

In [91]:
trainDF.groupBy("Age", "Gender").count().show()

+-----+------+------+
|  Age|Gender| count|
+-----+------+------+
|51-55|     F|  9894|
|18-25|     M| 75032|
| 0-17|     F|  5083|
|46-50|     M| 32502|
|18-25|     F| 24628|
|  55+|     M| 16421|
|  55+|     F|  5083|
|36-45|     M| 82843|
|26-35|     F| 50752|
| 0-17|     M| 10019|
|36-45|     F| 27170|
|51-55|     M| 28607|
|26-35|     M|168835|
|46-50|     F| 13199|
+-----+------+------+



#### Grouping with Expressions

In [92]:
trainDF.groupBy("Age").agg(
  count("Purchase").alias("quan"),
  expr("count(Purchase)")).show()

+-----+------+---------------+
|  Age|  quan|count(Purchase)|
+-----+------+---------------+
|18-25| 99660|          99660|
|26-35|219587|         219587|
| 0-17| 15102|          15102|
|46-50| 45701|          45701|
|51-55| 38501|          38501|
|36-45|110013|         110013|
|  55+| 21504|          21504|
+-----+------+---------------+



In [93]:
trainDF.groupBy("Age").agg(expr("avg(Purchase)"),expr("stddev_pop(Purchase)")).show()

+-----+-----------------+--------------------+
|  Age|    avg(Purchase)|stddev_pop(Purchase)|
+-----+-----------------+--------------------+
|18-25|9169.663606261289|   5034.296739627792|
|26-35|9252.690632869888|   5010.515894010163|
| 0-17|8933.464640444974|   5110.944823427662|
|46-50|9208.625697468327|   4967.162022122702|
|51-55|9534.808030960236|   5087.302011173864|
|36-45|9331.350694917874|   5022.901050378531|
|  55+|9336.280459449405|   5011.377469555773|
+-----+-----------------+--------------------+



In [94]:
## To find the mean of each age group in train dataset - Average purchases in each age group
trainDF.groupby('Age').agg({'Purchase': 'mean'}).show()

+-----+-----------------+
|  Age|    avg(Purchase)|
+-----+-----------------+
|18-25|9169.663606261289|
|26-35|9252.690632869888|
| 0-17|8933.464640444974|
|46-50|9208.625697468327|
|51-55|9534.808030960236|
|36-45|9331.350694917874|
|  55+|9336.280459449405|
+-----+-----------------+



In [95]:
trainDF.groupby('Age').agg({'Purchase': 'sum'}).show()

+-----+-------------+
|  Age|sum(Purchase)|
+-----+-------------+
|18-25|    913848675|
|26-35|   2031770578|
| 0-17|    134913183|
|46-50|    420843403|
|51-55|    367099644|
|36-45|   1026569884|
|  55+|    200767375|
+-----+-------------+



In [96]:
## Apply sum, min, max, count with groupby to get different summary insight for each group. 
exprs = {x: "sum" for x in trainDF.columns}
trainDF.groupBy("Age").agg(exprs).show()

+-----+------------------+-----------------------+-------------------+-------------+------------+---------------+-------------------------------+-----------------------+--------+-----------+-----------------------+---------------+
|  Age|sum(City_Category)|sum(Product_Category_3)|sum(Marital_Status)|sum(Purchase)|sum(User_ID)|sum(Occupation)|sum(Stay_In_Current_City_Years)|sum(Product_Category_1)|sum(Age)|sum(Gender)|sum(Product_Category_2)|sum(Product_ID)|
+-----+------------------+-----------------------+-------------------+-------------+------------+---------------+-------------------------------+-----------------------+--------+-----------+-----------------------+---------------+
|18-25|              null|               388041.0|              21116|    913848675| 99939196632|         671348|                       116997.0|                 509371|    null|       null|               654936.0|           null|
|26-35|              null|               846624.0|              86291|   203

## Joins

In [97]:
# in Python
person = spark.createDataFrame([
    (0, "Dr. Murthy", 0, [250, 100]),
    (1, "Dr. Sridhar Pappu", 1, [500, 250, 100]),
    (2, "Dr. Manoj", 2, [100])])\
  .toDF("id", "name", "graduate_program", "role_status")
graduateProgram = spark.createDataFrame([
    (0, "Ph.D", "School of Information", "Carnegie Mellon University"),
    (1, "Ph.D", "The University of Texas", "El Paso"),
    (2, "Ph.D.", "School of Information", "Oklahoma State University")])\
  .toDF("id", "degree", "department", "school")
roleStatus = spark.createDataFrame([
    (500, "President"),
    (250, "Founder"),
    (100, "Mentor")])\
  .toDF("id", "status")

In [98]:
person.show()

+---+-----------------+----------------+---------------+
| id|             name|graduate_program|    role_status|
+---+-----------------+----------------+---------------+
|  0|       Dr. Murthy|               0|     [250, 100]|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|
|  2|        Dr. Manoj|               2|          [100]|
+---+-----------------+----------------+---------------+



In [99]:
graduateProgram.show(truncate=False)

+---+------+-----------------------+--------------------------+
|id |degree|department             |school                    |
+---+------+-----------------------+--------------------------+
|0  |Ph.D  |School of Information  |Carnegie Mellon University|
|1  |Ph.D  |The University of Texas|El Paso                   |
|2  |Ph.D. |School of Information  |Oklahoma State University |
+---+------+-----------------------+--------------------------+



In [100]:
roleStatus.show()

+---+---------+
| id|   status|
+---+---------+
|500|President|
|250|  Founder|
|100|   Mentor|
+---+---------+



In [101]:
person.createOrReplaceTempView("personTbl")
graduateProgram.createOrReplaceTempView("graduateProgramTbl")
roleStatus.createOrReplaceTempView("roleStatusTbl")

#### Inner Joins

In [102]:
joinExpression = person["graduate_program"] == graduateProgram['id']
person.join(graduateProgram, joinExpression).show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  2|        Dr. Manoj|               2|          [100]|  2| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



In [103]:
spark.sql("""SELECT * FROM personTbl JOIN graduateProgramTbl
  ON personTbl.graduate_program = graduateProgramTbl.id""").show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  2|        Dr. Manoj|               2|          [100]|  2| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



In [104]:
joinType = "inner"
person.join(graduateProgram, joinExpression, joinType).show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  2|        Dr. Manoj|               2|          [100]|  2| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



In [105]:
spark.sql("""SELECT * FROM personTbl INNER JOIN graduateProgramTbl
  ON personTbl.graduate_program = graduateProgramTbl.id""").show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  2|        Dr. Manoj|               2|          [100]|  2| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



#### Outer Joins

In [106]:
joinType = "outer"
person.join(graduateProgram, joinExpression, joinType).show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  2|        Dr. Manoj|               2|          [100]|  2| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



In [107]:
spark.sql("""SELECT * FROM personTbl FULL OUTER JOIN graduateProgramTbl
  ON personTbl.graduate_program = graduateProgramTbl.id""").show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  2|        Dr. Manoj|               2|          [100]|  2| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



#### Left Outer Joins

In [108]:
joinType = "left_outer"
graduateProgram.join(person, joinExpression, joinType).show()

+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+
| id|degree|          department|              school| id|             name|graduate_program|    role_status|
+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+
|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|  0|       Dr. Murthy|               0|     [250, 100]|
|  1|  Ph.D|The University of...|             El Paso|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|
|  2| Ph.D.|School of Informa...|Oklahoma State Un...|  2|        Dr. Manoj|               2|          [100]|
+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+



In [109]:
spark.sql("""SELECT * FROM personTbl LEFT OUTER JOIN graduateProgramTbl
  ON personTbl.graduate_program = graduateProgramTbl.id""").show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  2|        Dr. Manoj|               2|          [100]|  2| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



#### Right Outer Joins

In [110]:
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  2|        Dr. Manoj|               2|          [100]|  2| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



In [111]:
spark.sql("""SELECT * FROM personTbl RIGHT OUTER JOIN graduateProgramTbl
  ON personTbl.graduate_program = graduateProgramTbl.id""").show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  2|        Dr. Manoj|               2|          [100]|  2| Ph.D.|School of Informa...|Oklahoma State Un...|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+



#### Natural Joins
Natural joins make implicit guesses at the columns on which you would like to join. 
It finds matching columns and returns the results. 
Left, right, and outer natural joins are all supported.

WARNING:
Implicit is always dangerous! 
The following query will give us incorrect results because 
the two DataFrames/tables share a column name (id), but it means different things in the datasets. 
You should always use this join with caution.

In [112]:
spark.sql("""SELECT * FROM graduateProgramTbl NATURAL JOIN personTbl""").show()

+---+------+--------------------+--------------------+-----------------+----------------+---------------+
| id|degree|          department|              school|             name|graduate_program|    role_status|
+---+------+--------------------+--------------------+-----------------+----------------+---------------+
|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|       Dr. Murthy|               0|     [250, 100]|
|  1|  Ph.D|The University of...|             El Paso|Dr. Sridhar Pappu|               1|[500, 250, 100]|
|  2| Ph.D.|School of Informa...|Oklahoma State Un...|        Dr. Manoj|               2|          [100]|
+---+------+--------------------+--------------------+-----------------+----------------+---------------+



#### Cross (Cartesian) Joins
Cross-joins in simplest terms are inner joins that do not specify a predicate. 
Cross joins will join every single row in the left DataFrame to ever single row in the right DataFrame. 
This will cause an absolute explosion in the number of rows contained in the resulting DataFrame. 
If you have 1,000 rows in each DataFrame, the cross-join of these will result in 1,000,000 (1,000 x 1,000) rows. 
For this reason, you must very explicitly state that you want a cross-join by using the cross join keyword:

In [113]:
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()

+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+
| id|degree|          department|              school| id|             name|graduate_program|    role_status|
+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+
|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|  0|       Dr. Murthy|               0|     [250, 100]|
|  1|  Ph.D|The University of...|             El Paso|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|
|  2| Ph.D.|School of Informa...|Oklahoma State Un...|  2|        Dr. Manoj|               2|          [100]|
+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+



In [114]:
spark.sql("""SELECT * FROM graduateProgramTbl CROSS JOIN personTbl
  ON graduateProgramTbl.id = personTbl.graduate_program""").show()

+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+
| id|degree|          department|              school| id|             name|graduate_program|    role_status|
+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+
|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|  0|       Dr. Murthy|               0|     [250, 100]|
|  1|  Ph.D|The University of...|             El Paso|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|
|  2| Ph.D.|School of Informa...|Oklahoma State Un...|  2|        Dr. Manoj|               2|          [100]|
+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+



In [115]:
person.crossJoin(graduateProgram).show()

+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
| id|             name|graduate_program|    role_status| id|degree|          department|              school|
+---+-----------------+----------------+---------------+---+------+--------------------+--------------------+
|  0|       Dr. Murthy|               0|     [250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  0|       Dr. Murthy|               0|     [250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  0|       Dr. Murthy|               0|     [250, 100]|  2| Ph.D.|School of Informa...|Oklahoma State Un...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  2|        Dr. Manoj|               2|          [100]|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|
|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|  1|  Ph.D|The University of...|             El Paso|
|  1|Dr. S

In [116]:
spark.sql("""SELECT * FROM graduateProgramTbl CROSS JOIN personTbl""").show()

+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+
| id|degree|          department|              school| id|             name|graduate_program|    role_status|
+---+------+--------------------+--------------------+---+-----------------+----------------+---------------+
|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|  0|       Dr. Murthy|               0|     [250, 100]|
|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|
|  0|  Ph.D|School of Informa...|Carnegie Mellon U...|  2|        Dr. Manoj|               2|          [100]|
|  1|  Ph.D|The University of...|             El Paso|  0|       Dr. Murthy|               0|     [250, 100]|
|  2| Ph.D.|School of Informa...|Oklahoma State Un...|  0|       Dr. Murthy|               0|     [250, 100]|
|  1|  Ph.D|The University of...|             El Paso|  1|Dr. Sridhar Pappu|               1|[500, 250, 100]|
|  1|  Ph.

#### Joins on Complex Types

In [117]:
from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personId")\
  .join(roleStatus, expr("array_contains(role_status, id)")).show()

+--------+-----------------+----------------+---------------+---+---------+
|personId|             name|graduate_program|    role_status| id|   status|
+--------+-----------------+----------------+---------------+---+---------+
|       0|       Dr. Murthy|               0|     [250, 100]|250|  Founder|
|       0|       Dr. Murthy|               0|     [250, 100]|100|   Mentor|
|       1|Dr. Sridhar Pappu|               1|[500, 250, 100]|500|President|
|       1|Dr. Sridhar Pappu|               1|[500, 250, 100]|250|  Founder|
|       1|Dr. Sridhar Pappu|               1|[500, 250, 100]|100|   Mentor|
|       2|        Dr. Manoj|               2|          [100]|100|   Mentor|
+--------+-----------------+----------------+---------------+---+---------+



In [118]:
spark.sql("""SELECT * FROM
  (select id as personId, name, graduate_program, role_status FROM personTbl)
  INNER JOIN roleStatusTbl ON array_contains(role_status, id)
""").show()

+--------+-----------------+----------------+---------------+---+---------+
|personId|             name|graduate_program|    role_status| id|   status|
+--------+-----------------+----------------+---------------+---+---------+
|       0|       Dr. Murthy|               0|     [250, 100]|250|  Founder|
|       0|       Dr. Murthy|               0|     [250, 100]|100|   Mentor|
|       1|Dr. Sridhar Pappu|               1|[500, 250, 100]|500|President|
|       1|Dr. Sridhar Pappu|               1|[500, 250, 100]|250|  Founder|
|       1|Dr. Sridhar Pappu|               1|[500, 250, 100]|100|   Mentor|
|       2|        Dr. Manoj|               2|          [100]|100|   Mentor|
+--------+-----------------+----------------+---------------+---+---------+



#### Random Samples

In [119]:
## To create a sample DataFrame from the base DataFrame
## Use sample operation to take sample of a DataFrame. 
## The sample method on DataFrame will return a DataFrame containing the sample of base DataFrame. 
## The sample method takes 3 parameters.
## withReplacement = True or False to select a observation with or without replacement.
## fraction = x, where x = .5 shows that we want to have 50% data in sample DataFrame.
## seed to reproduce the result
sampleDF1 = trainDF.sample(False, 0.2, 1234)
sampleDF2 = trainDF.sample(False, 0.2, 4321)
print(sampleDF1.count(), sampleDF2.count())

(110322, 109779)


#### Random Splits

In [120]:
splitDF = trainDF.randomSplit([0.7, 0.3], seed=1234)
print(splitDF[0].count())
print(splitDF[1].count())

385461
164607


#### Map Transformation

In [121]:
## To apply map operation on DataFrame columns
## Apply a function on each row of DataFrame using map operation. 
## After applying this function, we get the result in the form of RDD. 
## Apply a map operation on User_ID column of train and print the first 5 elements of mapped RDD(x,1) 
## ----- Applying lambda function.

trainDF.select('User_ID').rdd.map(lambda x:(x,1)).take(5)

[(Row(User_ID=1000001), 1),
 (Row(User_ID=1000001), 1),
 (Row(User_ID=1000001), 1),
 (Row(User_ID=1000001), 1),
 (Row(User_ID=1000002), 1)]

*__Prior to Spark 2.0, spark_df.map would alias to spark_df.rdd.map(). 
With Spark 2.0, you must explicitly call .rdd first.__*

#### Sorting Rows

In [122]:
## To sort the DataFrame based on column(s)
## Use orderBy operation on DataFrame to get sorted output based on some column. 
## The orderBy operation take two arguments.
## List of columns.
## ascending = True or False for getting the results in ascending or descending order(list in case of more than two columns )
## Sort the train DataFrame based on ‘Purchase’.
trainDF.orderBy(trainDF.Purchase.desc()).show(5)

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1003160| P00052842|     M|26-35|        17|            C|                         3|             0|                10|              15.0|              null|   23961|
|1002272| P00052842|     M|26-35|         0|            C|                         1|             0|                10|              15.0|              null|   23961|
|1001474| P00052842|     M|26-35|         4|            A|                         2|             1|                10|              15.0|              null|   23961

#### Repartition and Coalesce
Another important optimization opportunity is to partition the data according to some frequently filtered columns
which controls the physical layout of data across the cluster including the partitioning scheme and the number of
partitions.

Repartition will incur a full shuffle of the data, regardless of whether or not one is necessary. This means that you should typically only repartition when the future number of partitions is greater than your current number of
partitions or when you are looking to partition by a set of columns.

In [123]:
## Find existing partitions count
trainDF.rdd.getNumPartitions()
## Do the repartition
## trainDF.repartition(5)

## Repartition based on a column
## If we know we are going to be filtering by a certain column often, 
## it can be worth repartitioning based on that column.
## trainDF.repartition(col(“Purchase”))

## We can optionally specify the number of partitions we would like too.
## trainDF.repartition(5, col(“Purchase”))

## Coalesce on the other hand will not incur a full shuffle and will try to combine partitions. 
## This operation will shuffle our data into 5 partitions based on the Purchase, 
## then coalesce them (without a full shuffle).
## trainDF.repartition(5, col("Purchase")).coalesce(2)

2

## Miscellaneous

#### Unions

In [124]:
df1 = spark.createDataFrame([[1, 'Alex', 25],[3, 'Carol', 53],[5, 'Emily', 25],[7, 'Gabriel', 32],[9, 'Ilma', 35],[11, 'Kim', 45]], ['id', 'name', 'age'])
df2 = spark.createDataFrame([[2, 'Ben', 66],[4, 'Daniel', 28],[6, 'Frank', 64],[8, 'Harley', 29],[10, 'Jack', 35],[12, 'Litmya', 45]], ['id', 'name', 'age'])
print("Before")
print("DataFrame-1")
print(df1.show())
print("DataFrame-2")
print(df2.show())
print("After")
df1 = df1.union(df2)
print("DataFrame-1")
print(df1.show())

Before
DataFrame-1
+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|   Alex| 25|
|  3|  Carol| 53|
|  5|  Emily| 25|
|  7|Gabriel| 32|
|  9|   Ilma| 35|
| 11|    Kim| 45|
+---+-------+---+

None
DataFrame-2
+---+------+---+
| id|  name|age|
+---+------+---+
|  2|   Ben| 66|
|  4|Daniel| 28|
|  6| Frank| 64|
|  8|Harley| 29|
| 10|  Jack| 35|
| 12|Litmya| 45|
+---+------+---+

None
After
DataFrame-1
+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|   Alex| 25|
|  3|  Carol| 53|
|  5|  Emily| 25|
|  7|Gabriel| 32|
|  9|   Ilma| 35|
| 11|    Kim| 45|
|  2|    Ben| 66|
|  4| Daniel| 28|
|  6|  Frank| 64|
|  8| Harley| 29|
| 10|   Jack| 35|
| 12| Litmya| 45|
+---+-------+---+

None


#### Unions and condtional append

In [125]:
df1.union(df2).where("age < 60").show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|   Alex| 25|
|  3|  Carol| 53|
|  5|  Emily| 25|
|  7|Gabriel| 32|
|  9|   Ilma| 35|
| 11|    Kim| 45|
|  4| Daniel| 28|
|  8| Harley| 29|
| 10|   Jack| 35|
| 12| Litmya| 45|
|  4| Daniel| 28|
|  8| Harley| 29|
| 10|   Jack| 35|
| 12| Litmya| 45|
+---+-------+---+



In [126]:
## To add the new column in DataFrame
## Use withColumn operation to add new column (we can also replace) in base DataFrame and return a new DataFrame. 
## The withColumn operation will take 2 parameters.
## Column name to be added /replaced.
## Expression on column.

## Derive new column, ‘Purchase_new’ in train which is calculated by dviding Purchase column by 2.

trainDF.withColumn('Purchase_new', trainDF.Purchase /2.0).select('Purchase','Purchase_new').show(5)

+--------+------------+
|Purchase|Purchase_new|
+--------+------------+
|    8370|      4185.0|
|   15200|      7600.0|
|    1422|       711.0|
|    1057|       528.5|
|    7969|      3984.5|
+--------+------------+
only showing top 5 rows



In [127]:
## To drop a column in DataFrame
## To drop a column from the DataFrame use drop operation. 
## Drop the column called ‘Comb’ from the test and get the remaining columns in test dataframe
testDF.drop('Comb').columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3']

In [128]:
## To remove some categories of Product_ID column in test that are not present in Product_ID column in train
## Use an user defined function ( udf ) to remove the categories of a column which are in test but not in train.
## Calculate the categories in Product_ID column which are in test but not in train.
diff_cat_in_train_test=testDF.select('Product_ID').subtract(trainDF.select('Product_ID'))
diff_cat_in_train_test.count() # For distict count

5

In [129]:
diff_cat_in_train_test.show(2)

+----------+
|Product_ID|
+----------+
| P00074942|
| P00030342|
+----------+
only showing top 2 rows



In [130]:
## There are 46 different categories in test. 
## To remove these categories from the test ‘Product_ID’ column.

## Create the distinct list of categories called ‘not_found_cat’ from the diff_cat_in_train_test using map operation.
## Register a udf(user define function).
## User defined function will take each element of test column and search this in not_found_cat list and 
## it will put -1 ifit finds in this list otherwise it will do nothing.
not_found_cat = diff_cat_in_train_test.rdd.map(lambda x: x[0]).collect()
print(len(not_found_cat))
print(type(not_found_cat))
print(not_found_cat)

5
<type 'list'>
[u'P00074942', u'P00030342', u'P00279042', u'P00140842', u'P00058842']


#### User Defined Functions - UDF

In [131]:
## Register the udf, we need to import StringType from the pyspark.sql and udf from the pyspark.sql.functions. 
## The udf function takes 2 parameters as arguments:
## Return type (in my case StringType())
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

Function1 = udf(lambda x: '-1' if x in not_found_cat else x, StringType())

In [132]:
## In the above code function name is ‘Function1’ and we are putting ‘-1’  for not found catagories in test ‘Product_ID’. 
## Finally apply above ‘Function1’ function on test ‘Product_ID’ and take result in k for new column calles “NEW_Product_ID”.

k = testDF.withColumn("NEW_Product_ID",Function1(testDF["Product_ID"])).select('NEW_Product_ID')
k.where(k['NEW_Product_ID'] == -1).show(2)

+--------------+
|NEW_Product_ID|
+--------------+
|            -1|
|            -1|
+--------------+
only showing top 2 rows



In [133]:
## See the results by again calculating the different categories in k and train subtract operation.
diff_cat_in_train_test=k.select('NEW_Product_ID').subtract(trainDF.select('Product_ID'))
print(diff_cat_in_train_test.count())# For distinct count
print(diff_cat_in_train_test.distinct().count())# For distinct count

1
1


In [134]:
## The output 1 means we have now only 1 different category k and train.
diff_cat_in_train_test.distinct().collect()

[Row(NEW_Product_ID=u'-1')]

In [135]:
from pyspark.sql.functions import lit, round, bround
trainDF.select(round(lit("2.5")), bround(lit(2.5))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [136]:
spark.sql("SELECT round(2.5), bround(2.5)").show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|            3|             2|
+-------------+--------------+



In [137]:
from pyspark.sql.functions import corr
print(trainDF.stat.corr("Purchase", "Product_Category_1"))
trainDF.select(corr("Purchase", "Product_Category_1")).show()

-0.34370334592
+----------------------------------+
|corr(Purchase, Product_Category_1)|
+----------------------------------+
|              -0.34370334591990787|
+----------------------------------+



In [138]:
spark.sql("SELECT corr(Purchase, Product_Category_1) FROM trainDFTable").show()

+------------------------------------------------------------------+
|corr(CAST(Purchase AS DOUBLE), CAST(Product_Category_1 AS DOUBLE))|
+------------------------------------------------------------------+
|                                              -0.34370334591990787|
+------------------------------------------------------------------+



In [139]:
trainDF.stat.freqItems(["Age"]).show(truncate = False)

+----------------------------------------------+
|Age_freqItems                                 |
+----------------------------------------------+
|[18-25, 26-35, 55+, 46-50, 51-55, 36-45, 0-17]|
+----------------------------------------------+



#### String Manipulations

In [140]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim

trainDF.select(
ltrim(lit(" HELLO ")).alias("ltrim"),
rtrim(lit(" HELLO ")).alias("rtrim"),
trim(lit(" HELLO ")).alias("trim"),
lpad(lit("HELLO"), 3, " ").alias("lp"),
rpad(lit("HELLO"), 10, " ").alias("rp"))\
.show(2)

+------+------+-----+---+----------+
| ltrim| rtrim| trim| lp|        rp|
+------+------+-----+---+----------+
|HELLO | HELLO|HELLO|HEL|HELLO     |
|HELLO | HELLO|HELLO|HEL|HELLO     |
+------+------+-----+---+----------+
only showing top 2 rows



In [141]:
spark.sql("""SELECT
ltrim(' HELLLOOOO '),
rtrim(' HELLLOOOO '),
trim(' HELLLOOOO '),
lpad('HELLOOOO ', 3, ' '),
rpad('HELLOOOO ', 10, ' ')
FROM
trainDFTable""").show(2)

+------------------+------------------+-----------------+---------------------+----------------------+
|ltrim( HELLLOOOO )|rtrim( HELLLOOOO )|trim( HELLLOOOO )|lpad(HELLOOOO , 3,  )|rpad(HELLOOOO , 10,  )|
+------------------+------------------+-----------------+---------------------+----------------------+
|        HELLLOOOO |         HELLLOOOO|        HELLLOOOO|                  HEL|            HELLOOOO  |
|        HELLLOOOO |         HELLLOOOO|        HELLLOOOO|                  HEL|            HELLOOOO  |
+------------------+------------------+-----------------+---------------------+----------------------+
only showing top 2 rows



#### Regular Expressions

In [142]:
from pyspark.sql.functions import regexp_replace
regex_string = "F|M"

trainDF.select(
regexp_replace(col("Gender"), regex_string, "MALE_OR_FEMALE")
.alias("Gender_DECODE"),
col("Gender"))\
.show(2)

+--------------+------+
| Gender_DECODE|Gender|
+--------------+------+
|MALE_OR_FEMALE|     F|
|MALE_OR_FEMALE|     F|
+--------------+------+
only showing top 2 rows



In [143]:
spark.sql("""
SELECT
regexp_replace(Gender, 'F|M', 'MALE_OR_FEMALE') as
Gender_DECODE,
Gender
FROM
trainDFTable
""").show(2)

+--------------+------+
| Gender_DECODE|Gender|
+--------------+------+
|MALE_OR_FEMALE|     F|
|MALE_OR_FEMALE|     F|
+--------------+------+
only showing top 2 rows



In [144]:
from pyspark.sql.functions import translate
trainDF.select(
translate(col("Gender"), "FM", "01"),
col("Gender"))\
.show(10)

+-------------------------+------+
|translate(Gender, FM, 01)|Gender|
+-------------------------+------+
|                        0|     F|
|                        0|     F|
|                        0|     F|
|                        0|     F|
|                        1|     M|
|                        1|     M|
|                        1|     M|
|                        1|     M|
|                        1|     M|
|                        1|     M|
+-------------------------+------+
only showing top 10 rows



In [145]:
spark.sql("""
SELECT
translate(Gender, 'FM', '01'),
Gender
FROM
trainDFTable
""").show(10)

+-------------------------+------+
|translate(Gender, FM, 01)|Gender|
+-------------------------+------+
|                        0|     F|
|                        0|     F|
|                        0|     F|
|                        0|     F|
|                        1|     M|
|                        1|     M|
|                        1|     M|
|                        1|     M|
|                        1|     M|
|                        1|     M|
+-------------------------+------+
only showing top 10 rows



## Working with Date and Time

In [146]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
.withColumn("today", current_date())\
.withColumn("now", current_timestamp())
dateDF.show(truncate = False)

+---+----------+-----------------------+
|id |today     |now                    |
+---+----------+-----------------------+
|0  |2018-11-20|2018-11-20 23:40:37.835|
|1  |2018-11-20|2018-11-20 23:40:37.835|
|2  |2018-11-20|2018-11-20 23:40:37.835|
|3  |2018-11-20|2018-11-20 23:40:37.835|
|4  |2018-11-20|2018-11-20 23:40:37.835|
|5  |2018-11-20|2018-11-20 23:40:37.835|
|6  |2018-11-20|2018-11-20 23:40:37.835|
|7  |2018-11-20|2018-11-20 23:40:37.835|
|8  |2018-11-20|2018-11-20 23:40:37.835|
|9  |2018-11-20|2018-11-20 23:40:37.835|
+---+----------+-----------------------+



In [147]:
dateDF.createOrReplaceTempView("dateDFTable")
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [148]:
from pyspark.sql.functions import date_add, date_sub
dateDF.select(date_sub(col("today"), 5),date_add(col("today"), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2018-11-15|        2018-11-25|
+------------------+------------------+
only showing top 1 row



In [149]:
spark.sql("""
SELECT
date_sub(today, 5),
date_add(today, 5)
FROM
dateDFTable
""").show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2018-11-15|        2018-11-25|
+------------------+------------------+
only showing top 1 row



In [150]:
from pyspark.sql.functions import datediff, months_between, to_date
dateDF\
.withColumn("week_ago", date_sub(col("today"), 7))\
.select(datediff(col("week_ago"), col("today")))\
.show(1)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row



In [151]:
dateDF\
.select(
to_date(lit("2017-01-01")).alias("start"),
to_date(lit("2018-02-18")).alias("end"))\
.select(months_between(col("start"), col("end")))\
.show(1)

+--------------------------+
|months_between(start, end)|
+--------------------------+
|               -13.5483871|
+--------------------------+
only showing top 1 row



In [152]:
spark.sql("""
SELECT
to_date('2016-01-01'),
months_between('2016-01-01', '2017-01-01'),
datediff('2016-01-01', '2017-01-01')
FROM
dateDFTable
""").show(2)

+---------------------+----------------------------------------------------------------------------+------------------------------------------------------------+
|to_date('2016-01-01')|months_between(CAST(2016-01-01 AS TIMESTAMP), CAST(2017-01-01 AS TIMESTAMP))|datediff(CAST(2016-01-01 AS DATE), CAST(2017-01-01 AS DATE))|
+---------------------+----------------------------------------------------------------------------+------------------------------------------------------------+
|           2016-01-01|                                                                       -12.0|                                                        -366|
|           2016-01-01|                                                                       -12.0|                                                        -366|
+---------------------+----------------------------------------------------------------------------+------------------------------------------------------------+
only showing top 2 rows



In [153]:
from pyspark.sql.functions import to_date, lit
spark.range(5).withColumn("date", lit("2017-01-01"))\
.select(to_date(col("date")))\
.show()

+---------------+
|to_date(`date`)|
+---------------+
|     2017-01-01|
|     2017-01-01|
|     2017-01-01|
|     2017-01-01|
|     2017-01-01|
+---------------+



__WARNING__
<br>Spark will not throw an error if it cannot parse the date, it’ll just return null. This can be a bit tricky in larger pipelines because you may be expecting your data in one format and getting it in another. To illustrate, let’s take a look at the date format that has switched from year-month-day to year-day-month. Spark will fail to parse this date and silently return null instead.

In [154]:
### 2016-20-12 - year-day-month
### 2017-12-11 - year-month-day
dateDF.select(to_date(lit("2016-20-12")),to_date(lit("2017-12-11"))).show(1)

+---------------------+---------------------+
|to_date('2016-20-12')|to_date('2017-12-11')|
+---------------------+---------------------+
|                 null|           2017-12-11|
+---------------------+---------------------+
only showing top 1 row



In [155]:
from pyspark.sql.functions import unix_timestamp, from_unixtime

dateFormat = "yyyy-dd-MM"

cleanDateDF = spark.range(1)\
.select(to_date(unix_timestamp(lit("2017-12-11"), dateFormat)
.cast("timestamp"))\
.alias("date"),
to_date(unix_timestamp(lit("2017-20-12"), dateFormat)
.cast("timestamp"))\
.alias("date2"))

cleanDateDF.show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [156]:
cleanDateDF.createOrReplaceTempView("dateTable2")

spark.sql("""
SELECT
to_date(cast(unix_timestamp(date, 'yyyy-dd-MM') as timestamp)),
to_date(cast(unix_timestamp(date2, 'yyyy-dd-MM') as timestamp)),
to_date(date)
FROM
dateTable2
""").show()

+---------------------------------------------------------------------------+----------------------------------------------------------------------------+--------------------------+
|to_date(CAST(unix_timestamp(datetable2.`date`, 'yyyy-dd-MM') AS TIMESTAMP))|to_date(CAST(unix_timestamp(datetable2.`date2`, 'yyyy-dd-MM') AS TIMESTAMP))|to_date(datetable2.`date`)|
+---------------------------------------------------------------------------+----------------------------------------------------------------------------+--------------------------+
|                                                                 2017-11-12|                                                                  2017-12-20|                2017-11-12|
+---------------------------------------------------------------------------+----------------------------------------------------------------------------+--------------------------+



In [157]:
cleanDateDF\
.select(unix_timestamp(col("date"), dateFormat).cast("timestamp"))\
.show(truncate=False)

+---------------------------------------------------+
|CAST(unix_timestamp(date, yyyy-dd-MM) AS TIMESTAMP)|
+---------------------------------------------------+
|2017-11-12 00:00:00                                |
+---------------------------------------------------+



In [158]:
cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [159]:
cleanDateDF.filter(col("date2") > "'2017-12-12'").show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [160]:
textDF = spark.range(10).withColumn("Description", lit("This is long string"))
textDF.show()

+---+-------------------+
| id|        Description|
+---+-------------------+
|  0|This is long string|
|  1|This is long string|
|  2|This is long string|
|  3|This is long string|
|  4|This is long string|
|  5|This is long string|
|  6|This is long string|
|  7|This is long string|
|  8|This is long string|
|  9|This is long string|
+---+-------------------+



In [161]:
from pyspark.sql.functions import split
textDF.select(split(col("Description"), " ")).show(2)

+---------------------+
|split(Description,  )|
+---------------------+
| [This, is, long, ...|
| [This, is, long, ...|
+---------------------+
only showing top 2 rows



In [162]:
textDF.createOrReplaceTempView('textDFTable')

spark.sql("""
SELECT
split(Description, ' ')
FROM
textDFTable
""").show(2)

+---------------------+
|split(Description,  )|
+---------------------+
| [This, is, long, ...|
| [This, is, long, ...|
+---------------------+
only showing top 2 rows



## User-Defined Functions

In [163]:
udfExampleDF = spark.range(5).toDF("num")

def power3(double_value):
    return double_value ** 3

power3(2.0)

8.0

Once the function is created, we need to register them with Spark so that we can used
them on all of our worker machines. Spark will serialize the function on the driver, and transfer it over the network to all executor processes. This happens regardless of language.

<br>Once we go to use the function, there are essentially two different things that occur. If the function is written in Scala or Java then we can use that function within the JVM. This means there will be little performance penalty aside from the fact that we can’t take advantage of code generation capabilities that Spark has for built-in functions.

<br>If the function is written in Python, something quite different happens. 
Spark will start up a python process on the worker, serialize all of the data to a format that python can understand (remember it was in the JVM before), execute the function row by row on that data in the python process, before finally returning the results of the row operations to the JVM and Spark.

![UDF_Spark_Python](./Images/UDF_Spark_Python.png)

In [164]:
from pyspark.sql.functions import udf
power3udf = udf(power3)

In [165]:
from pyspark.sql.functions import col
udfExampleDF.select(power3udf(col("num"))).show()

+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
|          8|
|         27|
|         64|
+-----------+



### Handling Different Data Sources

There are variety of data sources that one can use out of the box aswell as the countless other sources built by the greater community.

<br> **Spark** has six “core” data sources and hundreds of external data sources written by the community.

-  CSV
-  JSON
-  Parquet
-  ORC
-  JDBC/ODBC Connections
-  Plain-text files

<br> As mentioned, Spark has numerous community-created data sources. Here’s just a small sample:
-  Cassandra
-  HBase
-  MongoDB
-  AWS Redshift
-  XML
-  And many many others.

**Read API Structure**
<br>DataFrameReader.format(...).option("key", "value").schema(...).load(...)
<br>After we have a DataFrame reader, we specify several values:
-  The format
-  The schema
-  The read mode
-  A series of options

*Ex. spark.read.format("csv")
<br>  .option("mode", "FAILFAST")
<br>  .option("inferSchema", "true")
<br>  .option("path", "path/to/file(s)")
<br>  .schema(someSchema)
<br>  .load()
*

** READ MODES **
-  permissive - Sets all fields to null when it encounters a corrupted record and places all corrupted records in a string column called _corrupt_record.
-  dropMalformed - Drops the row that contains malformed records
-  failFast - Fails immediately upon encountering malformed records
<br><br>The default is permissive.

** Write API Structure **
<br>We will use this format to write to all of our data sources. 
<br>format is optional because by default, Spark will use the **Parquet** format. 
<br>option, again, allows us to configure how to write out our given data. 
<br>PartitionBy, bucketBy, and sortBy work only for file-based data sources; 
<br>you can use them to control the specific layout of files at the destination.

<br> DataFrameWriter.format(...).option(...).partitionBy(...).bucketBy(...).sortBy(...).save()
<br> The foundation for writing data is quite similar to that of reading data. 
<br>Instead of the DataFrameReader, we have the DataFrameWriter. 
<br>Because we always need to write out some given data source, 
<br>we access the DataFrameWriter on a per-DataFrame basis via the write attribute:

<br>After we have a DataFrameWriter, we specify three values: the format, a series of options, and the save mode. 

<br>Example: 
<br>dataframe.write.format("csv")
<br>  .option("mode", "OVERWRITE")
<br>  .option("dateFormat", "yyyy-MM-dd")
<br>  .option("path", "path/to/file(s)")
<br>  .save()

** SAVE MODES **
-  append - Appends the output files to the list of files that already exist at that location
-  overwrite - Will completely overwrite any data that already exists there
-  errorIfExists - Throws an error and fails the write if data or files already exist at the specified location
-  ignore - If data or files exist at the location, do nothing with the current DataFrame

#### CSV

In [166]:
tcs_CSV_DF = spark.read.format("csv")\
.option("inferSchema", "true")\
.option("header", "true")\
.load("file:///home/insofe/IBM/Day03/TCS_NSE.csv")

In [167]:
tcs_CSV_DF.show(4)

+-------------------+----------+----------+----------+----------+---------+--------+
|               Date|      Open|      High|       Low|     Close|Adj Close|  Volume|
+-------------------+----------+----------+----------+----------+---------+--------+
|2004-08-25 00:00:00|149.837006|149.837006|122.375000|123.494003|67.378128|  136928|
|2004-08-26 00:00:00|124.000000|124.625000|121.912003|122.375000|66.767601|40443200|
|2004-08-27 00:00:00|122.800003|122.800003|119.820000|120.332001|65.652962|30646000|
|2004-08-30 00:00:00|121.237999|123.750000|120.625000|123.345001|67.296806|24465208|
+-------------------+----------+----------+----------+----------+---------+--------+
only showing top 4 rows



In [168]:
tcs_CSV_DF.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)



In [169]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import expr, col, column

tcs_CSV_DF = tcs_CSV_DF.select(col("Date").cast("date"), 
                     col("Open").cast("double"),
                     col("High").cast("double"),
                     col("Low").cast("double"),
                     col("Close").cast("double"),
                     col("Adj Close").cast("double"), 
                     col("Volume").cast("int"))

In [170]:
tcs_CSV_DF.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [171]:
tcs_CSV_DF = tcs_CSV_DF.withColumnRenamed("Adj Close", "Adj_Close")
tcs_CSV_DF = tcs_CSV_DF.withColumnRenamed("Date", "Stock_Date")

In [172]:
tcs_CSV_DF.write.format("json").mode("overwrite").save("file:///home/insofe/IBM/Day03/TCS_JSON/")

#### JSON Files
Those coming from the world of JavaScript are likely familiar with JavaScript Object Notation, or JSON, as it’s commonly called.

In [173]:
tcs_JSON_DF = spark.read.format("json")\
.option("inferSchema", "True")\
.load("file:///home/insofe/IBM/Day03/TCS_JSON/")

In [174]:
tcs_JSON_DF.show(4)

+---------+----------+----------+----------+----------+----------+--------+
|Adj_Close|     Close|      High|       Low|      Open|Stock_Date|  Volume|
+---------+----------+----------+----------+----------+----------+--------+
|67.378128|123.494003|149.837006|   122.375|149.837006|2004-08-25|  136928|
|66.767601|   122.375|   124.625|121.912003|     124.0|2004-08-26|40443200|
|65.652962|120.332001|122.800003|    119.82|122.800003|2004-08-27|30646000|
|67.296806|123.345001|    123.75|   120.625|121.237999|2004-08-30|24465208|
+---------+----------+----------+----------+----------+----------+--------+
only showing top 4 rows



#### Text Files
Spark also allows you to read in plain-text files. 
<br>Each line in the file becomes a record in the DataFrame. 
<br>It is then up to you to transform it accordingly. 
<br>As an example of how you would do this, 
<br>suppose that you need to parse some Apache log files to some more structured format, 
<br>or perhaps you want to parse some plain text for natural-language processing. 

In [175]:
tcs_TEXT_DF = spark.read.text("file:///home/insofe/IBM/Day03/TCS_NSE.csv")\
  .selectExpr("split(value, ',') as rows")

In [176]:
tcs_TEXT_DF.show(4)

+--------------------+
|                rows|
+--------------------+
|[Date, Open, High...|
|[2004-08-25, 149....|
|[2004-08-26, 124....|
|[2004-08-27, 122....|
+--------------------+
only showing top 4 rows



#### SQL Databases
<br>driver = "org.sqlite.JDBC"
<br>url = "jdbc:sqlite:" + path
<br>tablename = "flight_info"

<br>dbDataFrame = spark.read.format("jdbc").option("url", url)\
<br>  .option("dbtable", tablename).option("driver",  driver).load()

pgDF = spark.read.format("jdbc")\
<br>  .option("driver", "org.postgresql.Driver")\
<br>  .option("url", "jdbc:postgresql://database_server")\
<br>  .option("dbtable", "schema.tablename")\
<br>  .option("user", "username").option("password", "my-secret-password").load()

** Query Pushdown **
<br> Spark makes a best-effort attempt to filter data in the database itself before creating the DataFrame. 
<br> Only the intended columns are selected..

<br>Ex. dbDataFrame.select("Age").distinct()
<br> Above query will fetch only the "Age" column from the original database.

Spark can actually do better than this on certain queries. 
<br>For example, if we specify a filter on our DataFrame, Spark will push that filter down into the database. 
<br>Ex. dbDataFrame.filter("Age > 50").show()

### HIVE

#### Create HIVE Table for the previously created DataFrame

In [177]:
tcs_CSV_DF.write.mode("overwrite").saveAsTable("tcs_stocks")

In [178]:
spark.sql("SELECT * FROM tcs_stocks LIMIT 10").show()

+----------+----------+----------+----------+----------+---------+--------+
|Stock_Date|      Open|      High|       Low|     Close|Adj_Close|  Volume|
+----------+----------+----------+----------+----------+---------+--------+
|2004-08-25|149.837006|149.837006|   122.375|123.494003|67.378128|  136928|
|2004-08-26|     124.0|   124.625|121.912003|   122.375|66.767601|40443200|
|2004-08-27|122.800003|122.800003|    119.82|120.332001|65.652962|30646000|
|2004-08-30|121.237999|    123.75|   120.625|123.345001|67.296806|24465208|
|2004-08-31|123.311996|    123.75|     122.0|123.512001|67.387932|21194656|
|2004-09-01|    123.75|   124.375|122.949997|123.487999|67.374817|19935544|
|2004-09-02|123.737999|125.574997|    123.25|124.207001|67.767113|21356352|
|2004-09-03|    125.75|     137.5|123.794998|124.732002|68.053581| 9869856|
|2004-09-06|129.988007|129.988007|124.112999|124.357002|67.848961| 9038672|
|2004-09-07|   129.375|   129.375|   124.375|124.449997|67.899704| 5772232|
+----------+

In [179]:
spark.sql("SHOW TABLES LIKE 'trainDF*'").show(truncate = False)

+--------+------------+-----------+
|database|tableName   |isTemporary|
+--------+------------+-----------+
|        |traindftable|true       |
+--------+------------+-----------+



In [180]:
spark.sql("SHOW TABLES LIKE 'tcs*'").show(truncate = False)

+--------+----------+-----------+
|database|tableName |isTemporary|
+--------+----------+-----------+
|default |tcs_stocks|false      |
+--------+----------+-----------+



In [181]:
spark.sql("""SELECT 
YEAR(stock_date) AS year, 
WEEKOFYEAR(stock_date) AS week, 
Avg(open) AS avg_open, 
Avg(high) AS avg_high, 
Avg(low) AS avg_low, 
Avg(close) AS avg_close, 
Avg(volume) AS avg_volume 
FROM tcs_stocks 
GROUP BY YEAR(stock_date), WEEKOFYEAR(stock_date) 
ORDER BY year DESC, week DESC""").show(10)

+----+----+------------------+------------------+------------------+------------------+----------+
|year|week|          avg_open|          avg_high|           avg_low|         avg_close|avg_volume|
+----+----+------------------+------------------+------------------+------------------+----------+
|2018|  46|1904.1666666666667|1928.3000083333334|1893.9166666666667|1912.1333009999998| 1933389.0|
|2018|  45|1921.6666666666667|1942.2666423333333|1893.5833333333333|1910.9166260000002| 1894185.0|
|2018|  44|1890.7099852000003|1926.0399901999997|1862.2199951999999|      1910.6300048| 2474390.0|
|2018|  43|      1870.3900146|      1883.7600096|      1830.2399904|           1849.45| 2412084.6|
|2018|  42|           1949.25|     1961.36251825|1911.2625122499999|1938.6000060000001|2915389.75|
|2018|  41|            2049.2|      2065.3599608|      1994.8699706|2022.2400148000002| 4140080.4|
|2018|  40|2161.9500122500003|     2201.89996325|      2105.8000795|2146.4625244999997|3963643.25|
|2018|  39